In [1]:
import codecs
import glob
import math
import os
from decimal import Decimal
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
file_name = "data/input/XXX"
df = pd.read_table(file_name)

In [3]:
initial_key = "Summary of Optimized"
end_key = "---------------------------------------------------"
Eigenvalues = "Eigenvalues"
torsion1 = "d15"
torsion2 = "d13"
# other_value = "xxx"

add_value = -632.0

In [4]:
# initial_keyを含む行を抜き出す
initial_key_df = df[df[df.columns[0]].str.contains(initial_key, case=False)]
initial_key_index = initial_key_df.index[0]
df = df.iloc[initial_key_index:]
# initial_keyを含む行以降のend_keyを含む行を抜き出す
end_key_df = df[df[df.columns[0]].str.contains(end_key)]
# dfを更新しているのでここでの行番号は0から始まる
end_key_index = end_key_df.index[0] - initial_key_index
df = df.iloc[:end_key_index]

# カラム名をわかりやすく変更
df = df.rename(columns={df.columns[0]: 'Col_1'})

df_E = df[df[df.columns[0]].str.contains(Eigenvalues, case=False)]

Eigenvalues_list = []
for n in range(len(df_E)):
    list_n = df_E.iloc[n][0].split(' ')
    list_n = ''.join(list_n).split('--')[-1].split("-")[1:]
    list_n = [float(s) * -1 for s in list_n]
    for m in range(len(list_n)):
        Eigenvalues_list.append(list_n[m])

Eigenvalues_list = [n+add_value for n in Eigenvalues_list]

# d15を含む行を全て抜き出す
# torsion1_listへ格納

df_t1 = df[df[df.columns[0]].str.contains(torsion1, case=False)]
torsion1_list_str = []
for t1_value in df_t1.iloc[0][0].split(' '):
    if t1_value != '':
        torsion1_list_str.append(t1_value)

torsion1_list = []

for n in range(len(df_t1)):
    list_n = df_t1.iloc[n][0].split(' ')
    list_t1 = []
    for t1_value in list_n:
        if t1_value != '':
            list_t1.append(t1_value)
    list_t1 = list_t1[1:]
    for m in range(len(list_t1)):
        torsion1_list.append(list_t1[m])

# 数値化
torsion1_list = [float(s) for s in torsion1_list]

# d13を含む行を全て抜き出す
# torsion2_listへ格納
df_t2 = df[df[df.columns[0]].str.contains(torsion2, case=False)]
torsion2_list_str = []
for t2_value in df_t2.iloc[0][0].split(' '):
    if t2_value != '':
        torsion2_list_str.append(t2_value)

torsion2_list = []

for n in range(len(df_t2)):
    list_n = df_t2.iloc[n][0].split(' ')
    list_t2 = []
    for t2_value in list_n:
        if t2_value != '':
            list_t2.append(t2_value)
    list_t2 = list_t2[1:]
    for m in range(len(list_t2)):
        torsion2_list.append(list_t2[m])

# 数値化
torsion2_list = [float(s) for s in torsion2_list]

In [5]:
# 一つのdfにまとめる
all_list = list(zip(Eigenvalues_list, torsion1_list, torsion2_list))
df_all = pd.DataFrame(data=all_list, columns=[Eigenvalues, torsion1, torsion2])

# 表形式のdfにする
df_output = pd.DataFrame()
torsion1_unique_list = sorted(list(set(df_all[torsion1])))
for n in torsion1_unique_list:
    df_t1_n = df_all[df_all[torsion1]==n]
    df_t1_n = pd.DataFrame(data=list(df_t1_n[Eigenvalues]), index=list(df_t1_n[torsion2]), columns=[n]).T
    df_output = pd.concat([df_output, df_t1_n])

In [6]:
# 出力確認
df_output

,0.0,15.0,30.0,45.0,60.0,75.0,90.0,105.0,120.0,135.0,...,210.0,225.0,240.0,255.0,270.0,285.0,300.0,315.0,330.0,345.0
0.0,-632.14386,-632.14439,-632.14519,-632.14519,-632.14421,-632.14290,-632.14223,-632.14270,-632.14398,-632.14509,...,-632.14525,-632.14509,-632.14398,-632.14270,-632.14223,-632.14290,-632.14421,-632.14519,-632.14519,-632.14439
15.0,-632.14257,-632.14323,-632.14400,-632.14392,-632.14287,-632.14157,-632.14095,-632.14147,-632.14273,-632.14375,...,-632.14405,-632.14382,-632.14265,-632.14137,-632.14096,-632.14168,-632.14296,-632.14384,-632.14372,-632.14292
30.0,-632.13871,-632.13945,-632.14021,-632.14009,-632.13902,-632.13775,-632.13718,-632.13772,-632.13892,-632.13982,...,-632.14025,-632.13999,-632.13881,-632.13756,-632.13720,-632.13794,-632.13914,-632.13990,-632.13967,-632.13891
45.0,-632.13259,-632.13337,-632.13416,-632.13408,-632.13305,-632.13183,-632.13129,-632.13179,-632.13289,-632.13369,...,-632.13420,-632.13398,-632.13284,-632.13164,-632.13131,-632.13200,-632.13310,-632.13374,-632.13346,-632.13273
60.0,-632.12540,-632.12617,-632.12704,-632.12708,-632.12618,-632.12500,-632.12441,-632.12481,-632.12584,-632.12663,...,-632.12710,-632.12700,-632.12597,-632.12481,-632.12441,-632.12501,-632.12606,-632.12669,-632.12638,-632.12559
75.0,-632.11939,-632.12007,-632.12103,-632.12124,-632.12047,-632.11930,-632.11862,-632.11899,-632.12012,-632.12097,...,-632.12112,-632.12118,-632.12026,-632.11908,-632.11861,-632.11921,-632.12036,-632.12103,-632.12069,-632.11976
90.0,-632.11640,-632.11691,-632.11786,-632.11817,-632.11747,-632.11631,-632.11571,-632.11623,-632.11746,-632.11836,...,-632.11797,-632.11812,-632.11724,-632.11609,-632.11573,-632.11648,-632.11770,-632.11842,-632.11807,-632.11702
105.0,-632.11657,-632.11685,-632.11770,-632.11800,-632.11736,-632.11636,-632.11591,-632.11650,-632.11775,-632.11872,...,-632.11781,-632.11794,-632.11714,-632.11617,-632.11596,-632.11675,-632.11799,-632.11878,-632.11849,-632.11740
120.0,-632.11882,-632.11891,-632.11964,-632.11995,-632.11945,-632.11857,-632.11812,-632.11869,-632.11999,-632.12103,...,-632.11975,-632.11991,-632.11926,-632.11839,-632.11816,-632.11893,-632.12023,-632.12111,-632.12090,-632.11979
135.0,-632.12170,-632.12172,-632.12244,-632.12281,-632.12235,-632.12142,-632.12092,-632.12148,-632.12282,-632.12391,...,-632.12256,-632.12279,-632.12216,-632.12123,-632.12095,-632.12172,-632.12306,-632.12400,-632.12383,-632.12272


In [ ]:
# 問題なければ出力
# 文字化けするなら encoding='utf_8_sig' をつける
result_path = "data/output/result.csv"
df_output.to_csv("data/output/result.csv")